In [1]:
import numpy as np
from ray.rllib.agents import cql, sac

In [2]:
env = "Pendulum-v1"

In [3]:
# config = sac.DEFAULT_CONFIG.copy()
# config["env"] = env
# config["output"] = "./experience"
# sampling_agent = sac.SACTrainer(config=config)

In [4]:
# i = 0
# while True:
#     print(f"Iter {i}")
#     results = sampling_agent.train()
#     reward = results.get("episode_reward_mean")

#     if reward >= -500:
#         break

#     i += 1

In [5]:
config = cql.CQL_DEFAULT_CONFIG.copy()
config["num_workers"] = 0  # Run locally.
config["horizon"] = 200
config["soft_horizon"] = True
config["no_done_at_end"] = True
config["n_step"] = 3
config["bc_iters"] = 20000
config["clip_actions"] = False
config["normalize_actions"] = True
config["learning_starts"] = 256
config["rollout_fragment_length"] = 1
config["prioritized_replay"] = False
config["tau"] = 0.005
config["target_entropy"] = "auto"
config["Q_model"] = {
    "fcnet_hiddens": [256, 256],
    "fcnet_activation": "relu",
}
config["policy_model"] = {
    "fcnet_hiddens": [256, 256],
    "fcnet_activation": "relu",
}
config["optimization"] = {
    "actor_learning_rate": 3e-4,
    "critic_learning_rate": 3e-4,
    "entropy_learning_rate": 3e-4,
}
config["train_batch_size"] = 256
config["target_network_update_freq"] = 1
config["timesteps_per_iteration"] = 1000
data_file = "./experience"
# print("data_file={} exists={}".format(data_file, os.path.isfile(data_file)))
config["input"] = data_file
# config["log_level"] = "INFO"
config["env"] = "Pendulum-v1"

# Set up evaluation.
config["evaluation_num_workers"] = 1
config["evaluation_interval"] = 1
config["evaluation_duration"] = 10
# This should be False b/c iterations are very long and this would
# cause evaluation to lag one iter behind training.
config["evaluation_parallel_to_training"] = False
# Evaluate on actual environment.
config["evaluation_config"] = {"input": "sampler"}
config["framework"] = "torch"

# Check, whether we can learn from the given file in `num_iterations`
# iterations, up to a reward of `min_reward`.
num_iterations = 5
min_reward = -300

# Test for torch framework (tf not implemented yet).
trainer = cql.CQLTrainer(config=config)
learnt = False
for i in range(num_iterations):
    print(f"Iter {i}")
    results = trainer.train()
    eval_results = results.get("evaluation")
    print("... R={}".format(results["episode_reward_mean"]))
    print("... R={}".format(eval_results["episode_reward_mean"]))
    # Learn until some reward is reached on an actual live env.
    if results["episode_reward_mean"] >= min_reward:
        learnt = True
        break
# if not learnt:
#     raise ValueError(
#         "CQLTrainer did not reach {} reward from expert "
#         "offline data!".format(min_reward)
#     )

2022-01-24 05:51:17,453	WARNING deprecation.py:45 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
2022-01-24 05:51:17,480	WARNING json_reader.py:63 -- Treating input directory as glob patterns: ['/home/yoshiakira/ml-bot/offline_rl/experience/*.json', '/home/yoshiakira/ml-bot/offline_rl/experience/*.zip']
2022-01-24 05:51:22,421	WARNING deprecation.py:45 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!


Loaded 16200 batches (16200 ts) into the replay buffer, which has capacity 1000000.


(RolloutWorker pid=6667) 2022-01-24 05:51:28,074	WARNING deprecation.py:45 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
2022-01-24 05:51:28,101	INFO trainable.py:124 -- Trainable.setup took 11.002 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Iter 0
... R=nan
... R=-1071.500645140501
Iter 1
... R=nan
... R=-1325.5430553778046
Iter 2
... R=nan
... R=-1359.2794287110291
Iter 3
... R=nan
... R=-1158.6344841593325
Iter 4
... R=nan
... R=-1201.5456201004852


E0124 06:40:03.755474800   32405 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1642974003.755433100","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1642974003.755427900","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
E0124 09:20:23.795829100   20773 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1642983623.795787300","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1642983623.795783200","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
